[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1XB8BrlawBNd2omQxuUSXYnCUnPapiRIH#scrollTo=Gpe-kl12ajfH)

# Imports

In [20]:
import sys, gspread, os
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots
from google.colab import files


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Helper Code

In [6]:
def overperformer(row):

  if row["Democrat Over/Under Performance"] > 0 and row["party"] == 'Democrat' or row["Republican Over/Under Performance"] > 0 and row["party"] == 'Republican':
    return "Overperformer"

  elif row["Democrat Over/Under Performance"] < 0 and row["party"] == 'Democrat' or row["Republican Over/Under Performance"] < 0 and row["party"] == 'Republican':
    return "Underperformer"


class ComparisonPlotter:

  def __init__(self, comparison_col: str, topics=list, comparison_col_map: dict=None):
    """
    Input:
      comparison_col (str): The column name that contains indicates group membership
      comparison_col_map (dict): Mapping str onto numeric comparison_col values
      topics (list): List of str with the names of the columns to compare
    """

    self.comparison_col = comparison_col
    self.comparison_col_map = comparison_col_map
    self.topics = topics
    self.colors = px.colors.qualitative.D3


  def plot(self, df, plot_type: str, title: str,):
    """
    Input:
      df: Pandas DataFrame
      plot_type (str): One of either "dot" or "bar"
      title (str): Title of entire figure
    Returns:
      plotly fig
    """

    temp_df = df.copy()
    temp_df = temp_df.dropna(subset=[self.comparison_col])
    assert plot_type in ["dot", "bar"], "plot_type must be one in ['dot', 'bar']"

    revserse = True if self.comparison_col == "winner" else False

    if self.comparison_col_map:
      temp_df[self.comparison_col] = temp_df[self.comparison_col].map(self.comparison_col_map)
      assert temp_df[self.comparison_col].isna().sum() == 0

    if plot_type=="bar":

      fig = make_subplots(
          rows=2,
          cols=1,
          shared_xaxes=True,
          vertical_spacing=0.02,
          )

      # Difference
      fig.add_trace(
          self._plot_differences(temp_df, orientation="v"),
          row=2,
          col=1,
          )
      fig.update_yaxes(showticklabels=False)

      # Comparison
      for i, col in enumerate(sorted(temp_df[self.comparison_col].unique(), reverse=revserse)):

          fig.add_trace(
            self._plot_bar_side_by_side(temp_df, i, col),
            row=1,
            col=1,
            )

      fig.update_layout(
          height=1200,
          width=1600,
          title= dict(
              text=title,
              font=dict(size=28)
              ),
          barmode='group',
          bargap=0.5,
          bargroupgap=0.05,
          )
      fig.update_xaxes(tickfont_size=18, row=2, col=1)
      fig.update_yaxes(title_text="Average Spending", titlefont_size=22, tickfont_size=18, row=1, col=1)
      fig.update_yaxes(title_text="Net Spending", titlefont_size=22, tickfont_size=18, row=2, col=1)

      return fig

    elif plot_type=="dot":

      fig = make_subplots(
          rows=1,
          cols=2,
          shared_yaxes=True,
          vertical_spacing=0.02,
          horizontal_spacing = 0.01,
          )

      # Difference
      fig.add_trace(
          self._plot_differences(temp_df, orientation="h"),
          row=1,
          col=2,
          )

      # Comparison
      for i, col in enumerate(sorted(temp_df[self.comparison_col].unique(), reverse=revserse)):

          fig.add_trace(
            self._plot_dot(temp_df, i, col),
            row=1,
            col=1,
            )

      fig.update_layout(
          height=1200,
          width=1600,
          title= dict(
              text=title,
              font=dict(size=28)
              ),
        annotations=[
          go.layout.Annotation(
              showarrow=False,
              text="Data: Ad Observatory, Cook Partisan Voter Index, Edison Research for the National Election Pool via the BBC",
              font=dict(size=14),
              x=-0.05,
              y=-0.14,
              xref='paper',
              yref='paper'
        ),
        go.layout.Annotation(
              showarrow=False,
              text="NYU Cybersecurity for Democracy",
              font=dict(size=14, color="#57068c"),
              x=-0.05,
              y=-0.16,
              xref='paper',
              yref='paper'
        )],
        margin=dict(
                    b=150,
                    pad=4,
                    ),
          )
      fig.update_xaxes(title_text="Average Spending", titlefont_size=22, tickfont_size=18, row=1, col=1)
      fig.update_xaxes(title_text="Net Spending", titlefont_size=22, tickfont_size=18, row=1, col=2)
      fig.update_yaxes(tickfont_size=18, row=1, col=1, autorange=False, range=[-0.5, len(self.topics)-0.5])

      return fig


  def _plot_differences(self, df, orientation):

    ascending = False if orientation == "v" else True

    self.comparisons = []
    revserse = True if self.comparison_col == "winner" else False
    for i, col in enumerate(sorted(df[self.comparison_col].unique(), reverse=revserse)):

      self.comparisons.append(
          pd.DataFrame(df[df[self.comparison_col]==col][self.topics].mean().round()).rename(columns={0: "Average Spending"}).reset_index().rename(columns={0: "topic"})
          )

    comparison_df = pd.DataFrame(self.comparisons[0].set_index("topic") - self.comparisons[1].set_index("topic")).rename(columns={"Average Spending": "Net Spending"}).sort_values(by="Net Spending", ascending=ascending)
    comparison_df["Color"] = np.where(comparison_df["Net Spending"]<0, self.colors[3], self.colors[2])


    # Order side-by-side plot
    self.comparisons = [i.reset_index(drop=True)[["topic", "Average Spending"]].merge(comparison_df.reset_index(), on="topic", how="left").sort_values(by="Net Spending", ascending=ascending) for i in self.comparisons]

    x_vals = comparison_df.reset_index()['topic'] if orientation == 'v' else comparison_df['Net Spending']
    y_vals = comparison_df['Net Spending'] if orientation == 'v' else comparison_df.reset_index()['topic']


    return go.Bar(
        name='Net Spending',
        x=x_vals,
        y=y_vals,
        marker_color=comparison_df['Color'],
        orientation=orientation,
        showlegend=False,
          )

  def _plot_bar_side_by_side(self, df, i, col):

    return go.Bar(
        x=self.comparisons[i]["topic"],
        y=self.comparisons[i]["Average Spending"],
        name=col,
        marker_color=self.colors[i],
        )

  def _plot_dot(self, df, i, col,):

      return go.Scatter(
          x=self.comparisons[i]["Average Spending"],
          y=self.comparisons[i]["topic"],
          marker=dict(color=self.colors[i], size=12),
          mode="markers",
          name=col,
      )


def bar_plotter(df, x_col: str, y_col: str, color_col: str, title:str, min_thresh: int, save_name: str):

  df = df[df[y_col] > min_thresh]
  fig = px.bar(df, x=x_col, y=y_col, color=color_col, title=title)
  fig.update_layout(
      barmode='stack',
      xaxis={'categoryorder':'total descending'},
      font=dict(size=24,),
      annotations=[
          go.layout.Annotation(
              showarrow=False,
              text="Data: Ad Observatory",
              font=dict(size=14),
              x=-0,
              y=-0.6,
              xref='paper',
              yref='paper'
        ),
        go.layout.Annotation(
              showarrow=False,
              text="NYU Cybersecurity for Democracy",
              font=dict(size=14, color="#57068c"),
              x=-0,
              y=-0.66,
              xref='paper',
              yref='paper'
        )],
        margin=dict(
                    b=150,
                    pad=4,
                    ),
      )
  fig.update_xaxes(title_text="",)


  if not os.path.exists("visualizations"):
    os.mkdir("visualizations")

  return fig


def spend_victory_correlations(df, independent_vars: list, dependent_var: str, significance_level: float=0.05, group_name: str=None, critical_value_correction= None, fdr= None):

  import scipy.stats as stats
  import statsmodels as sm

  df = df.dropna(subset=[dependent_var])

  if critical_value_correction == "bonferroni":
    significance_level = significance_level / len(independent_vars)


  correlations = None
  if len(df[dependent_var].unique()) == 2:
    correlations = [stats.pointbiserialr(df[i], df[dependent_var]) for i in independent_vars]

  else:
    correlations = [stats.pearsonr(df[i], df[dependent_var]) for i in independent_vars]

  pvalues = [i[1] for i in correlations]
  correlations = [i[0] for i in correlations]


  sig_topics = None
  if critical_value_correction == "benjamini-hochberg":
    assert fdr, "Please provide an FDR when using a Benjamini-Hochberg correction"

    rejected, _ = sm.stats.multitest.fdrcorrection(pvalues)
    sig_topics = [independent_vars[i] for i in np.where(rejected == False)[0]]

  else:
    sig_topics = [topic for pval, topic in zip(pvalues, independent_vars) if pval < significance_level]

  print(f"{group_name} Statistically significant topics (confidence level of {significance_level} with {critical_value_correction} correction): {sig_topics}")

  stats_df = pd.DataFrame({"Topics": independent_vars, f"{group_name} Correlations": correlations})
  return stats_df

# Save/Load Checkpoint

In [ ]:
files.upload()

In [42]:
df = pd.read_excel("Congressional Winners.xlsx", sheet_name="full_data", index_col=0)

for i in ["Democrat Results", "Democrat Lean", "Democrat Over/Under Performance"]:
  df[i] = df[i].map(lambda x: None if not x else float(x))


In [4]:
df["Republican Over/Under Performance"] = -df["Democrat Over/Under Performance"]
df["Overperformer"] = df.apply(lambda row: overperformer(row), axis=1)

In [5]:
df.columns = df.columns.str.lower()
df = df.rename(columns={"trump": "Trump", "ukraine": "Ukraine", "lgbtq+": "LGBTQ+"})

# Analyze

In [17]:
topic_cols = ['abortion', 'addiction', 'dark money', 'democratic norms',
       'discrimination', 'disinformation', 'donation', 'economy', 'education',
       'election', 'environmental protection', 'foreign policy', 'guns',
       'healthcare', 'immigration', 'judicial branch', 'LGBTQ+',
       'latino community', 'law enforcement', 'military', 'national security',
       'Trump', 'Ukraine', 'uncategorized', 'total',]

just_topics = ['abortion', 'addiction', 'dark money', 'democratic norms',
       'discrimination', 'disinformation', 'economy', 'education',
       'election', 'environmental protection', 'foreign policy', 'guns',
       'healthcare', 'immigration', 'judicial branch', 'LGBTQ+',
       'latino community', 'law enforcement', 'military', 'national security',
       'Trump', 'Ukraine',]


cp_winner = ComparisonPlotter(
    comparison_col= "winner",
    comparison_col_map= {0: "Loser", 1: "Winner"},
    topics=just_topics,
)

cp_performer = ComparisonPlotter(
    comparison_col= "overperformer",
    topics=just_topics,
)

## RQ1: What did the Trump picks advertise about?

In [10]:
df_trump = df.query("trump_picks==1").reset_index(drop=True)

In [11]:
df_trump["total"].sum()

643065.3

In [12]:
fig = bar_plotter(
    pd.DataFrame(df_trump[just_topics].sum().round().sort_values(ascending=False)).rename(columns={0: "Spend"}).reset_index().rename(columns={0: "Topic"}),
    x_col= "Topic",
    y_col= "Spend",
    color_col= None,
    min_thresh=2000,
    title = "Trump-endorsed congressional candidate spending by topic (with spending > $2k since Jan 1, 2022)",
    save_name= "trump_endorsees_spending_by_topic"
    )
fig.show()

### Winnner/Loser

In [13]:
df_trump["winner"].value_counts()

0    11
1     7
Name: winner, dtype: int64

In [18]:
cp_winner.plot(
    df_trump,
    plot_type="bar",
    title= "Average Spending Disparity Between Trump-Endorsed Winners and Losers by Topic",
    ).show()

cp_winner.plot(
    df_trump,
    plot_type="dot",
    title= "Average Spending Disparity Between Trump-Endorsed Winners and Losers by Topic",
    ).show()

### By Over/Under Performer

In [ ]:
df_trump["overperformer"].value_counts()

Underperformer    11
Overperformer      6
Name: overperformer, dtype: int64

In [ ]:
fig = cp_performer.plot(
    df_trump,
    plot_type="dot",
    title= "Average apending disparity between Trump-endorsed congressional over/under performers by topic (since Jan 1, 2022)",
    )
fig.show()


'https://plotly.com/~austinbotelho/3/'

## RQ2: What did over-performing Dems advertise about? [DEPRECIATED]

In [ ]:
df_dems = df.query("key_dems == 1").reset_index(drop=True)

In [ ]:
df_dems["Total"].sum()

5199832.140000001

In [ ]:
bar_plotter(
    pd.DataFrame(df_dems[just_topics].sum().round().sort_values(ascending=False)).reset_index().rename(columns={"index": "Topic", 0: "Spend"}),
    x_col= "Topic",
    y_col= "Spend",
    color_col= None,
    min_thresh=5000,
    title = "Key Democrat Combined Spending by Topic",
    save_name= "key_dems_spending"
    ).show()

In [ ]:
df_dems['winner'].value_counts()

1    32
0     3
Name: winner, dtype: int64

In [ ]:
# dot_plotter(
#     df_dems,
#     comparison_col="winner",
#     comparison_col_map={0: "Loser", 1: "Winner"},
#     y_cols=just_topics,
#     title="Average Spending Disparity between Key Democrat Race Winners and Losers",
#     save_name="key_dems_winners_and_losers",
#     ).show()

cp_winner.plot(
    df_dems,
    plot_type="dot",
    title= "Average Spending Disparity Between Trump-Endorsed Winners and Losers by Topic",
    ).show()

['Winner' 'Loser']


## RQ3: Was there a correlation between total spent (or spend by category) and election victories?

### Total Spend

In [ ]:
df["total"].sum()

36097489.68

In [ ]:
df["party"]  = df["party"].replace(r'^\s*$', "Other", regex=True)

total_spend_df = pd.DataFrame()
for party in df["party"].unique():
  temp_df = pd.DataFrame(df.query("party == @party")[just_topics].sum().round(), columns=["Spending"])
  temp_df["Party"] = party
  total_spend_df = total_spend_df.append(temp_df)

total_spend_df = total_spend_df.reset_index().rename(columns={0: "Topic"})

In [ ]:
fig = bar_plotter(
    total_spend_df,
    x_col= "Topic",
    y_col= "Spending",
    color_col= "Party",
    min_thresh=0,
    title = "Total spending in congressional general elections by topic and party (since Jan 1, 2022)",
    save_name= "total_spending_by_topic_2022"
    )
fig.show()

'https://plotly.com/~austinbotelho/5/'

### Correlations

#### Point Biserial

In [ ]:
# Democrats
stats_df = spend_victory_correlations(df.query("party == 'Democrat'"), group_name="Democrat", dependent_var="winner").set_index("Topics")
stats_df["Democrat Correlations"] = stats_df["Democrat Correlations"].fillna(0)
stats_df = stats_df.sort_values(by="Democrat Correlations", ascending=False)

display(stats_df.style.background_gradient(cmap='Blues'))

Democrat Statistically significant topics:  []


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,Democrat Correlations
Topics,
Military,0.097847
Immigration,0.092660
National Security,0.068830
Environmental Protection,0.063976
Discrimination,0.061198
Healthcare,0.050860
Democratic Norms,0.042744
Dark Money,0.023931
Economy,0.013324


In [ ]:
# Republicans
stats_df = spend_victory_correlations(df.query("party == 'Republican'"), group_name="Republican", dependent_var="winner").sort_values(by="Republican Correlations", ascending=False).set_index("Topics")

display(stats_df.style.background_gradient(cmap='Blues'))

Republican Statistically significant topics:  ['Immigration', 'Judicial Branch']


,Republican Correlations
Topics,
Immigration,0.142876
Judicial Branch,0.108320
Democratic Norms,0.104326
Trump,0.100011
Abortion,0.097585
Healthcare,0.071794
Military,0.059154
Addiction,0.055120
Economy,0.035427


#### Pearson

In [ ]:
# Democrats
stats_df = spend_victory_correlations(
    df.query("party == 'Democrat'"),
    group_name="Democrat",
    independent_vars= topic_cols,
    dependent_var='democrat over/under performance',
    critical_value_correction=False,
    ).set_index("Topics")
stats_df["Democrat Correlations"] = stats_df["Democrat Correlations"].fillna(0)
stats_df = stats_df.sort_values(by="Democrat Correlations", ascending=False)

display(stats_df.style.background_gradient(cmap='Blues'))

Democrat Statistically significant topics (confidence level of 0.05 with False correction): ['immigration']


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,Democrat Correlations
Topics,
immigration,0.144784
environmental protection,0.097640
military,0.062696
economy,0.053241
education,0.039356
national security,0.035126
healthcare,0.032779
discrimination,0.032675
dark money,0.023649


In [ ]:
# Republicans
stats_df = spend_victory_correlations(
    df.query("party == 'Republican'"),
    group_name="Republican",
    independent_vars= topic_cols,
    dependent_var='republican over/under performance',
    bonferroni_correction=True).set_index("Topics")
stats_df["Republican Correlations"] = stats_df["Republican Correlations"].fillna(0)
stats_df = stats_df.sort_values(by="Republican Correlations", ascending=False)

display(stats_df.style.background_gradient(cmap='Blues'))

Republican Statistically significant topics (confidence level of 0.002): []


,Republican Correlations
Topics,
judicial branch,0.127564
immigration,0.112784
abortion,0.085496
economy,0.077778
trump,0.076680
democratic norms,0.076333
healthcare,0.058970
uncategorized,0.054784
total,0.039256


### Compare Winner and Losers

In [ ]:
# Average Spending by Party by Result
df[["party", "winner", "total"]].groupby(["party", "winner"]).mean()

0                           total
party       winner               
Democrat    0       122190.703581
            1        67821.035988
Independent 0          733.427143
            1         1657.020000
Libertarian 0         1609.855714
Other       0          571.333333
            1          450.000000
Republican  0        19763.502327
            1        20018.843782

In [ ]:
# Average Spending by Party by Result
df[["party", "overperformer", "total"]].groupby(["party", "overperformer"]).mean()

0                                  total
party      overperformer                
Democrat   Overperformer   106971.919643
           Underperformer   71303.018661
Republican Overperformer    19733.846173
           Underperformer   20345.671642

#### Democrats

In [ ]:
df_dems = df.query("party == 'Democrat'").reset_index(drop=True)

In [ ]:
len(df_dems)

310

In [ ]:
df_dems["total"].sum().round()

29071232.0

In [ ]:
fig = bar_plotter(
    pd.DataFrame(df_dems[just_topics].sum().round().sort_values(ascending=False)).rename(columns={0: "Spending"}).reset_index().rename(columns={0: "Topic"}),
    x_col= "Topic",
    y_col= "Spending",
    color_col= None,
    min_thresh=200000,
    title = "Congressional Democrat combined spending by topic (with spend > $200k since Jan 1, 2022)",
    save_name= "dems_spending"
    )

fig.show()
py.plot(fig, filename="democrat_spending_by_topic_2022", auto_open=True)


'https://plotly.com/~austinbotelho/7/'

##### By Winnner/Loser

In [ ]:
df_dems['winner'].value_counts()

1    162
0    148
Name: winner, dtype: int64

In [ ]:
cp_winner.plot(
    df_dems,
    plot_type="bar",
    title= "Average Spending Disparity Between Democrat Winners and Losers by Topic",
    ).show()

cp_winner.plot(
    df_dems,
    plot_type="dot",
    title= "Average Spending Disparity Between Democrat Winners and Losers by Topic",
    ).show()

##### By Over/Under Performer

In [ ]:
df_dems['overperformer'].value_counts()

Overperformer     168
Underperformer    127
Name: Overperformer, dtype: int64

In [ ]:
fig = cp_performer.plot(
    df_dems,
    plot_type="dot",
    title= "Average spending disparity between Democrat over/under performers by topic (since Jan 1, 2022)",
    )

fig.show()

'https://plotly.com/~austinbotelho/9/'

#### Republicans

In [ ]:
df_reps = df.query("party == 'Republican'").reset_index(drop=True)

In [ ]:
len(df_reps)

352

In [ ]:
df_reps["total"].sum().round()

7006034.0

In [ ]:
fig = bar_plotter(
    pd.DataFrame(df_reps[just_topics].sum().round().sort_values(ascending=False)).rename(columns={0: "Spending"}).reset_index().rename(columns={0: "Topic"}),
    x_col= "Topic",
    y_col= "Spending",
    color_col= None,
    min_thresh=50000,
    title = "Congressional Republican combined spending by topic (with spend > $50k since Jan 1, 2022)",
    save_name= "reps_spending"
    )

fig.show()

'https://plotly.com/~austinbotelho/13/'

##### By Winnner/Loser

In [ ]:
df_reps['winner'].value_counts()

1    193
0    159
Name: winner, dtype: int64

In [ ]:
cp_winner.plot(
    df_reps,
    plot_type="bar",
    title= "Average Spending Disparity Between Republican Winners and Losers by Topic",
    ).show()

cp_winner.plot(
    df_reps,
    plot_type="dot",
    title= "Average Spending Disparity Between Republican Winners and Losers by Topic",
    ).show()

##### By Over/Under Performer

In [ ]:
df_reps['overperformer'].value_counts()

Overperformer     196
Underperformer    134
Name: Overperformer, dtype: int64

In [ ]:
len(df_reps)

352

In [ ]:

fig = cp_performer.plot(
    df_reps,
    plot_type="dot",
    title= "Average apending disparity between congressional Republican over/under performers by topic (since Jan 1, 2022)",
    )

fig.show()

'https://plotly.com/~austinbotelho/15/'